In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obsessive-Compulsive_Disorder"
cohort = "GSE60190"

# Input paths
in_trait_dir = "../../input/GEO/Obsessive-Compulsive_Disorder"
in_cohort_dir = "../../input/GEO/Obsessive-Compulsive_Disorder/GSE60190"

# Output paths
out_data_file = "../../output/preprocess/Obsessive-Compulsive_Disorder/GSE60190.csv"
out_gene_data_file = "../../output/preprocess/Obsessive-Compulsive_Disorder/gene_data/GSE60190.csv"
out_clinical_data_file = "../../output/preprocess/Obsessive-Compulsive_Disorder/clinical_data/GSE60190.csv"
json_path = "../../output/preprocess/Obsessive-Compulsive_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Optional, Dict, Any, Callable

# Load the necessary data to analyze
# Check all files in the directory to find which ones might contain clinical data
all_files = os.listdir(in_cohort_dir)
print(f"Files in directory: {all_files}")

# Check for gene data availability (look for files that might contain gene expression data)
gene_data_files = [f for f in all_files if "_gene" in f or "gene" in f.lower() or "expression" in f.lower()]
is_gene_available = len(gene_data_files) > 0
print(f"Gene data files: {gene_data_files}")

# Let's look for clinical data in any available text files
clinical_data_files = [f for f in all_files if f.endswith('.txt')]
print(f"Potential clinical data files: {clinical_data_files}")

clinical_data = None
if clinical_data_files:
    # Try to open each file to find clinical data
    for file in clinical_data_files:
        try:
            file_path = os.path.join(in_cohort_dir, file)
            df = pd.read_csv(file_path, sep='\t', index_col=0)
            
            # If file has relevant data structure, it's likely our clinical data
            if df.shape[0] > 0 and df.shape[1] > 0:
                print(f"Found potential clinical data in file: {file}")
                print(f"Data shape: {df.shape}")
                print(f"Sample columns: {list(df.columns)[:5]}")
                print(f"Sample index entries: {list(df.index)[:5]}")
                
                # Display sample values to help identify trait and gender information
                for idx in df.index[:10]:  # Look at first 10 rows
                    unique_vals = df.loc[idx].unique()
                    print(f"Row {idx}: {unique_vals}")
                
                clinical_data = df
                break
        except Exception as e:
            print(f"Could not read {file} as a data file: {e}")

# Based on inspection of the clinical data, determine the correct rows and conversion functions
if clinical_data is not None:
    # Determine trait row - look for information about OCD status
    trait_row = None
    gender_row = None
    
    # Look through the rows to find OCD and gender information
    for idx in clinical_data.index:
        row_values = [str(val).lower() for val in clinical_data.loc[idx].values if not pd.isna(val)]
        row_str = ' '.join(row_values)
        
        # Check for trait information
        if ('ocd' in row_str or 'obsessive' in row_str or 'control' in row_str or 
            'patient' in row_str or 'disorder' in row_str):
            trait_row = idx
            print(f"Found potential trait row at index {idx}")
            print(f"Sample values: {clinical_data.loc[idx].head()}")
        
        # Check for gender information
        if ('gender' in row_str or 'sex' in row_str or 'male' in row_str or 'female' in row_str):
            gender_row = idx
            print(f"Found potential gender row at index {idx}")
            print(f"Sample values: {clinical_data.loc[idx].head()}")
else:
    print("No clinical data found in any of the files.")

# Define age row (assuming it's not available based on initial analysis)
age_row = None

# Define conversion functions for each variable
def convert_trait(value):
    """Convert OCD status to binary (0 for control, 1 for OCD)."""
    if pd.isna(value):
        return None
    
    # Extract value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if not isinstance(value, str):
        value = str(value)
    
    value = value.lower()
    if 'control' in value or 'healthy' in value or 'normal' in value:
        return 0
    elif 'ocd' in value or 'obsessive' in value or 'patient' in value or 'disorder' in value:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to continuous value."""
    # Function for completeness, but not used since age data is likely not available
    if pd.isna(value):
        return None
    
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    if pd.isna(value):
        return None
    
    # Extract value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if not isinstance(value, str):
        value = str(value)
    
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# Save metadata for initial filtering
is_trait_available = trait_row is not None and clinical_data is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if is_trait_available:
    try:
        selected_clinical_df = geo_select_clinical_features(
            clinical_data, 
            trait, 
            trait_row, 
            convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical data:")
        print(preview)
        
        # Check if we have valid data (not all NaN)
        has_valid_data = selected_clinical_df.notna().any().any()
        if not has_valid_data:
            print("WARNING: Extracted clinical data contains only NaN values. Check row identifiers and conversion functions.")
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical features: {e}")
        # Update trait availability in case of extraction error
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=False
        )
else:
    print("Clinical data extraction skipped: No trait data available.")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 5: Gene Identifier Review

In [ ]:
# The gene identifiers start with "ILMN_" which indicates they are Illumina probe IDs 
# rather than human gene symbols. Illumina probes need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# Check if there are any platforms defined in the SOFT file that might contain annotation data
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for platform sections in the SOFT file
platform_sections = re.findall(r'^!Platform_title\s*=\s*(.+)$', soft_content, re.MULTILINE)
if platform_sections:
    print(f"Platform title found: {platform_sections[0]}")

# Try to extract more annotation data by reading directly from the SOFT file
# Look for lines that might contain gene symbol mappings
symbol_pattern = re.compile(r'ID_REF\s+Symbol|ID\s+Gene Symbol', re.IGNORECASE)
annotation_lines = []
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if symbol_pattern.search(line):
            annotation_lines.append(line)
            # Collect the next few lines to see the annotation structure
            for _ in range(10):
                annotation_lines.append(next(f, ''))

if annotation_lines:
    print("Found potential gene symbol mappings:")
    for line in annotation_lines:
        print(line.strip())

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("\nGene annotation preview:")
print(preview_df(gene_annotation, n=10))

# If we need an alternative source of mapping, check if there are any other annotation files in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
annotation_files = [f for f in cohort_files if 'annotation' in f.lower() or 'platform' in f.lower()]
if annotation_files:
    print("\nAdditional annotation files found in the cohort directory:")
    for file in annotation_files:
        print(file)